In [17]:
from langchain_core.messages import BaseMessage,HumanMessage,SystemMessage
from langgraph.graph import StateGraph
from langgraph.prebuilt import ToolNode,tools_condition
from langgraph.graph.message import add_messages
from typing import Annotated, Sequence
from langchain.tools import tool

from langchain_google_genai import ChatGoogleGenerativeAI

from typing import Annotated,Union
from pydantic import BaseModel,Field

In [2]:
from langchain.utilities import SQLDatabase

# Point to your local movies.db
# db = SQLDatabase.from_uri("sqlite:///imdb_data.db")
# db = SQLDatabase.from_uri(r"sqlite:///D:\Programming\Python in Sublime\LLM Works\LangGraph Basic\Graphs\Voice Agents\chinook\chinook.db")
db = SQLDatabase.from_uri(r"sqlite:///D:\Programming\Python in Sublime\LLM Works\LangGraph Basic\Graphs\Voice Agents\databases\Car_Database.db")

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
class AgentState(BaseModel):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [5]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=os.getenv("GOOGLE_API_KEY"))

### Tools

In [6]:
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
db_tools = toolkit.get_tools()

In [7]:
llm_with_tools = llm.bind_tools(db_tools)

In [8]:
from langchain import hub
prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")
system_message = prompt_template.format(dialect="SQLite", top_k=5)

## can be use for builtin prompt for improves accuracy of SQL query generation.

In [9]:
def agent_node(state: AgentState) -> AgentState:
    # system_prompt = SystemMessage(content=
    #     "You are my AI assistant, please answer my query to the best of your ability."
    # )
    system_prompt = SystemMessage(content=system_message)
    response = llm_with_tools.invoke([system_prompt]+state.messages)
    return {"messages": [response]}

In [10]:
graph = StateGraph(AgentState)

graph.add_node(node="db_agent",action=agent_node)
graph.add_node(node="tools",action=ToolNode(tools=db_tools))

graph.set_entry_point(key="db_agent")

graph.add_conditional_edges(source="db_agent",path=tools_condition)

graph.add_edge(start_key="tools",end_key="db_agent")
graph.set_finish_point(key="db_agent")

app = graph.compile()

In [16]:
inputs = {
    "messages": 
    [HumanMessage(
        content="""Names of the tables well formatted output like 1. 2. 3. soo on""")]
}

result = app.invoke(inputs)
result["messages"][-1].pretty_print()

================================== Ai Message ==================================

1. Brands
2. Car_Options
3. Car_Parts
4. Car_Vins
5. Customer_Ownership
6. Customers
7. Dealer_Brand
8. Dealers
9. Manufacture_Plant
10. Models


In [ ]:
# for message_chunk, metadata in app.stream(inputs, stream_mode="messages"):
#     if message_chunk.content:
#         print(message_chunk.content, end=" ", flush=True)
#     # message["messages"][-1].pretty_print()

In [ ]:
from IPython.display import display,Image
display(Image(app.get_graph().draw_mermaid_png()))